# Python 设计模式教程

欢迎来到 Python 设计模式教程！设计模式是软件开发中针对常见问题的经过验证的、可重用的解决方案。它们不是具体的代码或库，而是一种描述问题和解决方案之间关系的方式，可以帮助你编写更灵活、可维护和可扩展的代码。

Python 是一种动态且富有表现力的语言，其许多特性（如函数是一等公民、动态类型、鸭子类型）使得某些设计模式的实现比在静态类型语言中更简洁或有所不同。

**为什么学习设计模式？**

1.  **提供通用词汇**：开发者之间可以更有效地沟通设计思想。
2.  **重用经验证的解决方案**：避免重复造轮子，解决常见设计问题。
3.  **提高代码质量**：使代码更易于理解、修改和维护。
4.  **促进代码复用**：模式本身就是可复用的思想。

**设计模式分类 (GoF - Gang of Four 经典分类)：**

*   **创建型模式 (Creational Patterns)**：关注对象的创建机制，旨在以适合特定情况的方式创建对象。
*   **结构型模式 (Structural Patterns)**：关注类和对象的组合，形成更大的结构。
*   **行为型模式 (Behavioral Patterns)**：关注对象之间的职责分配和算法的抽象。

本教程将介绍一些常见的设计模式及其在 Python 中的 Pythonic 实现。

## 1. 创建型模式 (Creational Patterns)

### 1.1 工厂方法 (Factory Method)

**意图**：定义一个创建对象的接口，但让子类决定实例化哪一个类。工厂方法使一个类的实例化延迟到其子类。

**Pythonic 实现**：在 Python 中，由于函数是一等公民，工厂方法通常可以直接用一个返回不同类型实例的函数或静态方法来实现，不一定需要严格的类继承结构。

In [ ]:
from abc import ABC, abstractmethod

# 产品接口
class Document(ABC):
    @abstractmethod
    def open(self):
        pass

# 具体产品
class TextDocument(Document):
    def open(self):
        print("Opening text document...")
        return "Text Content"

class PdfDocument(Document):
    def open(self):
        print("Opening PDF document...")
        return "PDF Content"

# 创建者 (工厂) - 经典方式
class DocumentFactory(ABC):
    @abstractmethod
    def create_document(self) -> Document:
        pass

    def process_document(self):
        doc = self.create_document()
        content = doc.open()
        print(f"Processing content: {content}")

# 具体创建者
class TextDocumentFactory(DocumentFactory):
    def create_document(self) -> Document:
        return TextDocument()

class PdfDocumentFactory(DocumentFactory):
    def create_document(self) -> Document:
        return PdfDocument()

print("--- Classic Factory Method ---")
text_factory = TextDocumentFactory()
text_factory.process_document()

pdf_factory = PdfDocumentFactory()
pdf_factory.process_document()

# Pythonic 工厂函数
def create_document_pythonic(doc_type: str) -> Document:
    if doc_type == 'text':
        return TextDocument()
    elif doc_type == 'pdf':
        return PdfDocument()
    else:
        raise ValueError(f"Unknown document type: {doc_type}")

print("\n--- Pythonic Factory Function ---")
doc1 = create_document_pythonic('text')
print(doc1.open())
doc2 = create_document_pythonic('pdf')
print(doc2.open())

### 1.2 抽象工厂 (Abstract Factory)

**意图**：提供一个接口，用于创建一系列相关或相互依赖的对象，而无需指定它们具体的类。

**场景**：当你的系统需要独立于其产品的创建、组合和表示时，或者当你有多个产品系列，并且希望在任何给定时间只使用一个系列时。

In [ ]:
# 抽象产品 A
class Button(ABC):
    @abstractmethod
    def paint(self):
        pass

# 抽象产品 B
class Checkbox(ABC):
    @abstractmethod
    def paint(self):
        pass

# 具体产品 (Windows 主题)
class WinButton(Button):
    def paint(self):
        print("Painting Windows button")

class WinCheckbox(Checkbox):
    def paint(self):
        print("Painting Windows checkbox")

# 具体产品 (Mac 主题)
class MacButton(Button):
    def paint(self):
        print("Painting Mac button")

class MacCheckbox(Checkbox):
    def paint(self):
        print("Painting Mac checkbox")

# 抽象工厂
class GUIFactory(ABC):
    @abstractmethod
    def create_button(self) -> Button:
        pass

    @abstractmethod
    def create_checkbox(self) -> Checkbox:
        pass

# 具体工厂 (Windows)
class WinFactory(GUIFactory):
    def create_button(self) -> Button:
        return WinButton()

    def create_checkbox(self) -> Checkbox:
        return WinCheckbox()

# 具体工厂 (Mac)
class MacFactory(GUIFactory):
    def create_button(self) -> Button:
        return MacButton()

    def create_checkbox(self) -> Checkbox:
        return MacCheckbox()

# 客户端代码
def create_ui(factory: GUIFactory):
    button = factory.create_button()
    checkbox = factory.create_checkbox()
    button.paint()
    checkbox.paint()

print("--- Abstract Factory ---")
print("Client: Creating UI with Windows theme")
create_ui(WinFactory())

print("\nClient: Creating UI with Mac theme")
create_ui(MacFactory())

### 1.3 单例模式 (Singleton)

**意图**：确保一个类只有一个实例，并提供一个全局访问点来访问该实例。

**Pythonic 实现**：在 Python 中，由于模块本身在导入时就是单例的，所以一种常见的做法是使用模块级变量。如果确实需要类实现单例，可以使用元类、装饰器或覆盖 `__new__` 方法。

**争议**：单例模式有时被认为是反模式，因为它引入了全局状态，可能使测试和代码理解变得困难。应谨慎使用。

In [ ]:
# 方法1: 使用元类
class SingletonMeta(type):
    _instances = {}
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            instance = super().__call__(*args, **kwargs)
            cls._instances[cls] = instance
        return cls._instances[cls]

class Logger(metaclass=SingletonMeta):
    def __init__(self, filename):
        # 注意：__init__ 仍然会在每次尝试创建时被调用，即使返回的是现有实例
        # 因此，初始化逻辑可能需要特殊处理，确保只执行一次
        if not hasattr(self, 'filename'): # 简单检查避免重复初始化
            print(f"Logger (metaclass) initialized with {filename}")
            self.filename = filename
    
    def log(self, message):
        print(f"LOG [{self.filename}]: {message}")

print("--- Singleton with Metaclass ---")
logger1 = Logger("app.log")
logger2 = Logger("another.log") # __init__ 会被调用，但返回的是 logger1
logger1.log("Event 1")
logger2.log("Event 2") # 使用的还是 app.log
print(f"logger1 is logger2: {logger1 is logger2}")
print(f"logger1.filename: {logger1.filename}") # 仍是 app.log

# 方法2: 使用装饰器 (更简洁，但 __init__ 行为类似)
def singleton_decorator(cls):
    instances = {}
    def get_instance(*args, **kwargs):
        if cls not in instances:
            instances[cls] = cls(*args, **kwargs)
        return instances[cls]
    return get_instance

@singleton_decorator
class DatabaseConnection:
    def __init__(self, dsn):
        if not hasattr(self, 'dsn'): # 简单检查避免重复初始化
            print(f"DatabaseConnection (decorator) initialized with {dsn}")
            self.dsn = dsn
            self.connection_id = id(self) # 只是为了演示
    
    def query(self, sql):
        print(f"Querying on {self.dsn} (ID: {self.connection_id}): {sql}")

print("\n--- Singleton with Decorator ---")
db1 = DatabaseConnection("user:pass@host1/db")
db2 = DatabaseConnection("user:pass@host2/db")
db1.query("SELECT * FROM users")
db2.query("SELECT * FROM products") # 使用的还是 host1
print(f"db1 is db2: {db1 is db2}")

# 方法3: 模块级单例 (最 Pythonic 的方式之一)
# my_config.py
class AppConfig:
    def __init__(self):
        self.settings = {}
        print("AppConfig (module level) initialized")
    def set(self, key, value):
        self.settings[key] = value
    def get(self, key):
        return self.settings.get(key)

# config_instance = AppConfig() # 在模块中创建实例
# # 在其他地方: from my_config import config_instance
print("\n--- Singleton with Module ---")
class ModuleSingletonSimulation: # 模拟模块导入行为
    _instance = None
    @classmethod
    def get_instance(cls):
        if cls._instance is None:
            cls._instance = AppConfig()
        return cls._instance

config1 = ModuleSingletonSimulation.get_instance()
config2 = ModuleSingletonSimulation.get_instance()
config1.set("theme", "dark")
print(f"config2 theme: {config2.get('theme')}")
print(f"config1 is config2: {config1 is config2}")

## 2. 结构型模式 (Structural Patterns)

### 2.1 适配器模式 (Adapter)

**意图**：将一个类的接口转换成客户希望的另一个接口。适配器使原本由于接口不兼容而不能一起工作的那些类可以一起工作。

**场景**：当你希望使用某个类，但是其接口与其他代码不兼容时。

In [ ]:
# 目标接口 (Target)
class ModernLogger:
    def log_message(self, level: str, message: str):
        print(f"MODERN LOG [{level.upper()}]: {message}")

# 被适配者 (Adaptee) - 一个老的、接口不同的日志库
class LegacyLogger:
    def write_log(self, text: str, severity_code: int):
        # severity_code: 1=INFO, 2=WARNING, 3=ERROR
        severity_map = {1: "INFO", 2: "WARN", 3: "ERR"}
        print(f"LEGACY LOG ({severity_map.get(severity_code, 'UNK')}): {text}")

# 适配器 (Adapter)
class LoggerAdapter(ModernLogger):
    def __init__(self, legacy_logger: LegacyLogger):
        self._legacy_logger = legacy_logger

    def log_message(self, level: str, message: str):
        # 将 ModernLogger 的接口调用转换为 LegacyLogger 的接口调用
        level_map = {"INFO": 1, "WARNING": 2, "ERROR": 3}
        severity_code = level_map.get(level.upper(), 1) # 默认为 INFO
        self._legacy_logger.write_log(message, severity_code)

print("--- Adapter Pattern ---")
legacy_logger_instance = LegacyLogger()
adapter = LoggerAdapter(legacy_logger_instance)

# 客户端现在可以使用 ModernLogger 接口与 LegacyLogger 交互
adapter.log_message("info", "This is an informational message.")
adapter.log_message("ERROR", "A critical error occurred!")

### 2.2 装饰器模式 (Decorator) - 结构型

**意图**：动态地给一个对象添加一些额外的职责。就增加功能来说，装饰器模式相比生成子类更为灵活。

**注意**：这里的“装饰器模式”是 GoF 设计模式中的一种结构型模式，它与 Python 语言特性中的“装饰器”(@-syntax) 虽然名称相同且有相似之处（都是包装），但概念上是独立的。Python 装饰器语法是实现装饰器模式的一种非常自然的方式。

**场景**：当你希望在不影响其他对象的情况下，以动态、透明的方式给单个对象添加职责。

In [ ]:
# 组件接口 (Component)
class Coffee(ABC):
    @abstractmethod
    def get_cost(self) -> float:
        pass

    @abstractmethod
    def get_description(self) -> str:
        pass

# 具体组件 (Concrete Component)
class SimpleCoffee(Coffee):
    def get_cost(self) -> float:
        return 5.0

    def get_description(self) -> str:
        return "Simple coffee"

# 装饰器基类 (Decorator Base)
class CoffeeDecorator(Coffee):
    def __init__(self, wrapped_coffee: Coffee):
        self._wrapped_coffee = wrapped_coffee

    @abstractmethod
    def get_cost(self) -> float:
        pass

    @abstractmethod
    def get_description(self) -> str:
        pass

# 具体装饰器 (Concrete Decorators)
class MilkDecorator(CoffeeDecorator):
    def get_cost(self) -> float:
        return self._wrapped_coffee.get_cost() + 1.5

    def get_description(self) -> str:
        return self._wrapped_coffee.get_description() + ", milk"

class SugarDecorator(CoffeeDecorator):
    def get_cost(self) -> float:
        return self._wrapped_coffee.get_cost() + 0.5

    def get_description(self) -> str:
        return self._wrapped_coffee.get_description() + ", sugar"

print("--- Decorator Pattern (Structural) ---")
my_coffee = SimpleCoffee()
print(f"Coffee: {my_coffee.get_description()}, Cost: ${my_coffee.get_cost():.2f}")

# 加牛奶
my_coffee = MilkDecorator(my_coffee)
print(f"Coffee: {my_coffee.get_description()}, Cost: ${my_coffee.get_cost():.2f}")

# 加糖
my_coffee = SugarDecorator(my_coffee)
print(f"Coffee: {my_coffee.get_description()}, Cost: ${my_coffee.get_cost():.2f}")

# 再加一份牛奶 (可以多次装饰)
my_coffee_double_milk = MilkDecorator(MilkDecorator(SimpleCoffee()))
print(f"Coffee: {my_coffee_double_milk.get_description()}, Cost: ${my_coffee_double_milk.get_cost():.2f}")

### 2.3 外观模式 (Facade)

**意图**：为子系统中的一组接口提供一个统一的、高层的接口。外观定义了一个高层接口，使得子系统更容易使用。

**场景**：当你想为一个复杂的子系统提供一个简单的接口时，或者你想将子系统与客户端解耦。

In [ ]:
# 复杂的子系统组件
class CPU:
    def freeze(self):
        print("CPU: Freezing...")
    def jump(self, position):
        print(f"CPU: Jumping to {position}...")
    def execute(self):
        print("CPU: Executing...")

class Memory:
    def load(self, position, data):
        print(f"Memory: Loading data '{data}' to position {position}...")

class HardDrive:
    def read(self, lba, size):
        print(f"HardDrive: Reading {size} bytes from LBA {lba}...")
        return "boot_data"

# 外观 (Facade)
class ComputerFacade:
    def __init__(self):
        self.cpu = CPU()
        self.memory = Memory()
        self.hard_drive = HardDrive()

    def start(self):
        print("\n--- ComputerFacade: Starting computer ---")
        self.cpu.freeze()
        boot_address = 0x0000
        boot_sector_data = self.hard_drive.read(lba=0, size=512)
        self.memory.load(boot_address, boot_sector_data)
        self.cpu.jump(boot_address)
        self.cpu.execute()
        print("--- ComputerFacade: Computer started ---")

print("--- Facade Pattern ---")
computer = ComputerFacade()
computer.start() # 客户端只需要调用一个简单的方法

## 3. 行为型模式 (Behavioral Patterns)

### 3.1 策略模式 (Strategy)

**意图**：定义一系列算法，把它们一个个封装起来，并且使它们可以相互替换。策略模式使得算法可独立于使用它的客户而变化。

**Pythonic 实现**：由于函数是一等公民，策略可以直接是函数，而不是必须是实现了特定接口的类。

In [ ]:
from typing import List, Callable

# 策略接口 (经典方式，Python中常直接用Callable)
class SortStrategy(ABC):
    @abstractmethod
    def sort(self, data: List[int]) -> List[int]:
        pass

# 具体策略
class BubbleSortStrategy(SortStrategy):
    def sort(self, data: List[int]) -> List[int]:
        print("Sorting using Bubble Sort")
        arr = list(data) # 创建副本以避免修改原始列表
        n = len(arr)
        for i in range(n):
            for j in range(0, n - i - 1):
                if arr[j] > arr[j + 1]:
                    arr[j], arr[j + 1] = arr[j + 1], arr[j]
        return arr

class QuickSortStrategy(SortStrategy):
    def sort(self, data: List[int]) -> List[int]:
        print("Sorting using Quick Sort (simplified)")
        arr = list(data)
        # 实际的快速排序实现会更复杂，这里用内置排序代替简化
        arr.sort() 
        return arr

# 上下文 (Context)
class Sorter:
    def __init__(self, strategy: SortStrategy = None):
        self._strategy = strategy or QuickSortStrategy() # 默认策略

    def set_strategy(self, strategy: SortStrategy):
        self._strategy = strategy

    def sort_data(self, data: List[int]) -> List[int]:
        return self._strategy.sort(data)

print("--- Strategy Pattern (Classic) ---")
data_to_sort = [5, 1, 4, 2, 8]
sorter = Sorter() # 默认使用 QuickSort
print(f"Sorted (default): {sorter.sort_data(data_to_sort)}")

sorter.set_strategy(BubbleSortStrategy())
print(f"Sorted (bubble): {sorter.sort_data(data_to_sort)}")

# Pythonic 策略 (使用函数)
def bubble_sort_func(data: List[int]) -> List[int]:
    print("Sorting using Bubble Sort (function)")
    # ... (实现同上)
    arr = list(data)
    n = len(arr)
    for i in range(n):
        for j in range(0, n - i - 1):
            if arr[j] > arr[j + 1]:
                arr[j], arr[j + 1] = arr[j + 1], arr[j]
    return arr

def quick_sort_func(data: List[int]) -> List[int]:
    print("Sorting using Quick Sort (function)")
    arr = list(data)
    arr.sort()
    return arr

class PythonicSorter:
    def __init__(self, strategy_func: Callable[[List[int]], List[int]] = quick_sort_func):
        self._strategy_func = strategy_func

    def set_strategy(self, strategy_func: Callable[[List[int]], List[int]]):
        self._strategy_func = strategy_func

    def sort_data(self, data: List[int]) -> List[int]:
        return self._strategy_func(data)

print("\n--- Strategy Pattern (Pythonic Functions) ---")
py_sorter = PythonicSorter()
print(f"Sorted (default func): {py_sorter.sort_data(data_to_sort)}")

py_sorter.set_strategy(bubble_sort_func)
print(f"Sorted (bubble func): {py_sorter.sort_data(data_to_sort)}")

### 3.2 观察者模式 (Observer)

**意图**：定义对象间的一种一对多的依赖关系，当一个对象的状态发生改变时，所有依赖于它的对象都得到通知并自动更新。

**场景**：当一个对象的改变需要同时改变其他对象，而且你不知道具体有多少对象需要改变时。或者当一个对象应该能通知其他对象，而不需要知道那些对象是谁时。

In [ ]:
# 主题 (Subject / Observable)
class Subject:
    def __init__(self):
        self._observers: List[Callable] = [] # 存储观察者 (回调函数)
        self._state = None

    def attach(self, observer: Callable):
        if observer not in self._observers:
            self._observers.append(observer)

    def detach(self, observer: Callable):
        try:
            self._observers.remove(observer)
        except ValueError:
            pass # Observer not found

    def notify(self):
        print("Subject: Notifying observers...")
        for observer in self._observers:
            observer(self._state) # 传递状态给观察者

    @property
    def state(self):
        return self._state

    @state.setter
    def state(self, new_state):
        print(f"Subject: State changed to {new_state}")
        self._state = new_state
        self.notify()

# 观察者 (可以是函数或实现了特定接口的类)
def concrete_observer_A(state):
    print(f"Observer A: Received update, new state is {state}")

class ConcreteObserverB:
    def __init__(self, name):
        self.name = name

    def update(self, state):
        print(f"Observer {self.name}: Received update, new state is {state}")

print("--- Observer Pattern ---")
subject = Subject()

observer_b_instance = ConcreteObserverB("B1")

subject.attach(concrete_observer_A)
subject.attach(observer_b_instance.update) # 传递方法作为回调

subject.state = "NEW_DATA_1"
print("-")
subject.state = "MORE_DATA_2"

subject.detach(concrete_observer_A)
print("-")
subject.state = "FINAL_DATA_3"

### 3.3 命令模式 (Command)

**意图**：将一个请求封装为一个对象，从而使你可用不同的请求对客户进行参数化；对请求排队或记录请求日志，以及支持可撤销的操作。

**场景**：当你想把发出请求的对象与执行请求的对象解耦，或者想支持撤销/重做操作、事务等。

In [ ]:
# 命令接口 (Command)
class Command(ABC):
    @abstractmethod
    def execute(self):
        pass

    # 可选的撤销方法
    # @abstractmethod
    # def undo(self):
    #     pass

# 接收者 (Receiver) - 实际执行操作的对象
class Light:
    def __init__(self, location: str):
        self.location = location
        self.is_on = False

    def turn_on(self):
        self.is_on = True
        print(f"Light in {self.location} turned ON")

    def turn_off(self):
        self.is_on = False
        print(f"Light in {self.location} turned OFF")

# 具体命令 (Concrete Commands)
class LightOnCommand(Command):
    def __init__(self, light: Light):
        self._light = light

    def execute(self):
        self._light.turn_on()

class LightOffCommand(Command):
    def __init__(self, light: Light):
        self._light = light

    def execute(self):
        self._light.turn_off()

# 调用者 (Invoker) - 持有命令并请求执行
class RemoteControl:
    def __init__(self):
        self._command: Command = None

    def set_command(self, command: Command):
        self._command = command

    def press_button(self):
        if self._command:
            print("Remote: Button pressed")
            self._command.execute()
        else:
            print("Remote: No command set")

print("--- Command Pattern ---")
living_room_light = Light("Living Room")
kitchen_light = Light("Kitchen")

light_on_living = LightOnCommand(living_room_light)
light_off_living = LightOffCommand(living_room_light)
light_on_kitchen = LightOnCommand(kitchen_light)

remote = RemoteControl()

remote.set_command(light_on_living)
remote.press_button()

remote.set_command(light_off_living)
remote.press_button()

remote.set_command(light_on_kitchen)
remote.press_button()

## 总结

设计模式为构建健壮、可维护的软件提供了宝贵的指导。

*   **创建型模式** 帮助你以灵活的方式创建对象。
*   **结构型模式** 帮助你组合对象形成更大的结构。
    * Python 的动态性和鸭子类型有时可以简化结构型模式的实现，例如，对于适配器，有时只需要一个简单的包装函数或类，而不必严格遵循接口继承。
*   **行为型模式** 帮助你管理对象之间的交互和职责分配。
    * Python 的一等函数特性使得许多行为型模式（如策略、命令、观察者中的回调）可以非常简洁地实现。

**学习设计模式的关键：**

1.  **理解意图**：每个模式解决什么问题？
2.  **了解结构**：模式中涉及哪些角色（类或对象）以及它们如何交互？
3.  **思考适用场景**：什么时候应该使用这个模式？什么时候不应该？
4.  **Pythonic 实现**：思考如何利用 Python 的特性（如动态类型、函数式特性、模块等）来更优雅地实现这些模式。

不要试图一次性记住所有模式，而是要在实际开发中遇到类似问题时，能够识别出可能适用的模式，并查阅相关资料。最好的学习方式是通过实践和阅读优秀的代码。